## Betingade sannolikheter

En betingade sannolikheter är ett mått sannolikheten givet att något annat har hänt. Vi betecknar detta
$$Pr(A|B) = \frac{Pr(A \cap B)}{Pr(B)}$$
och utrycker det som _Sannolikheten för A givet B_.


In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/student-alcohol-consumption/student-mat.csv')
df.head()   # Show first 5 rows of the dataframe

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [12]:
number_of_rows = len(df) # Number of rows in the dataframe
print(f'Number of students (represented by rows) in the dataframe: {number_of_rows}')

Number of students (represented by rows) in the dataframe: 395


Vi kommer bara titta på hur frånvaro (absences) och slutbetyg (G3, med 0-20 poäng) förhåller sig till varandra. 

Låt oss hitta alla med betyg A, dvs. 80% eller högre på slutprovet.

In [13]:
df['grade_A'] = np.where(df['G3']*5 >= 80, 1, 0) # Create a new column with 1 if the grade is A and 0 otherwise

Nu vill vi hitta alla med hög frånvaro och lägga i en boolean vektor

In [14]:
df['high_absenses'] = np.where(df['absences'] >= 10, 1, 0) # Create a new column with 1 if the absences is 10 or more and 0 otherwise

# Add a count column to the dataframe
df['count'] = 1

Ta bort alla columner som vi inte är intresserade av.

In [15]:
df = df[['grade_A', 'high_absenses', 'count']]
df.head(7)

,grade_A,high_absenses,count
0,0,0,1
1,0,0,1
2,0,1,1
3,0,0,1
4,0,0,1
5,0,1,1
6,0,0,1


Nu kan vi skapa en pivot tabell som visar antalet förekomster i varje kategori.

In [16]:
# Group by grade_A and high_absenses and sum the count column to create pivot table
pd.pivot_table(
    df, 
    values='count', 
    index=['grade_A'], 
    columns=['high_absenses'], 
    aggfunc=np.size, 
    fill_value=0
) 

high_absenses,0,1
grade_A,,
0,277,78
1,35,5


Låt oss definiera sannolikheter som vi är intresserade av 
* `Pr(A)` : Sannolikheten för A
* `Pr(B)` : Sannolikheten att minst 10 föreläsningar
* `Pr(A|B)` : Sannolikheten att få A i slutbetyg förutsatt att studeten missat minst 10 föreläsningar

In [17]:
# Total number of students
nbr_students = len(df)

# Get probability
Pr_A = np.sum(df['grade_A']) / nbr_students
Pr_B = np.sum(df['high_absenses']) / nbr_students

print(f"Probability of getting an A: {Pr_A * 100:.2f}%")
print(f"Probability of having 10 or more absenses: {Pr_B * 100:.2f}%")


Probability of getting an A: 10.13%
Probability of having 10 or more absenses: 21.01%


Då $Pr(A|B) = \frac{Pr(A \cap B)}{Pr(B)}$ behöver vi få fram $Pr(A \cap B)$. Detta är detsamma som sannolikheten för **_Både A och B_**

In [18]:
# Get probability of getting an A and having 10 or more absenses
Pr_AnB = np.sum(df['grade_A'] & df['high_absenses']) / nbr_students
print(f"Probability of getting an A and having 10 or more absenses: {Pr_AnB * 100:.2f}%")

Probability of getting an A and having 10 or more absenses: 1.27%


Nu har vi allt för att svara på frågan: "Hur sannolikt är det att få A när man missat minst 10 föreläsningar"

In [20]:
# Get probability of getting an A given that the student has 10 or more absenses
Pr_AB = Pr_AnB / Pr_B
print(f"Probability of getting an A given that you have 10 or more absenses: {Pr_AB * 100:.2f}%")

Probability of getting an A given that you have 10 or more absenses: 6.02%


Lägg märket till den subtila språkliga skillnaden mellan **Givet** och **Och** i formuleringen av sannolikheterna!